# Exercise PPM Part 2 - Small dataset

Imports

In [1]:
import pandas as pd
import warnings
import numpy as np
from datetime import datetime
import calendar
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
warnings.simplefilter("ignore")

Import an event log as a pandas dataframe

In [2]:
df = pd.read_csv("Event_Logs/small3insert.csv")
df

,Case,Event,Activity,Timestamp,Resource,Resource_failure_rate,Resource_Pass/Fail,order,resource_anomaly_type,resource_parameter,trace_change_resource,variant_num
0,case_0,9367,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,0.026984,0,1,normal,NaN,0,var_358
1,case_0,9368,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,0.018454,0,2,normal,NaN,0,var_358
2,case_0,9369,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,0.002551,0,3,normal,NaN,0,var_358
3,case_0,9370,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,0.029775,0,4,normal,NaN,0,var_358
4,case_0,9371,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_358
...,...,...,...,...,...,...,...,...,...,...,...,...
45842,case_999,31364,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,0.018454,0,5,normal,NaN,0,var_171
45843,case_999,31365,Activity K,1970-01-01 14:00:00,Resource_Group1_res_1,0.007068,0,6,normal,NaN,0,var_171
45844,case_999,31366,Activity Q,1970-01-01 15:00:00,Resource_Group0_res_0,0.018454,0,7,normal,NaN,0,var_171
45845,case_999,31367,Activity L,1970-01-01 16:00:00,Resource_Group0_res_0,0.018454,0,8,normal,NaN,0,var_171


Create an event log that retains only the attributes caseid, activity, timestamp, resource and outcome.

In [3]:
df = df.rename(columns={"Case":"caseid", "Activity":"activity", "Timestamp":"ts", "Resource":"resource", "resource_anomaly_type":"outcome"})
df_2 = df[["caseid", "activity", "ts", "resource", "outcome"]].copy()
df_2

,caseid,activity,ts,resource,outcome
0,case_0,Activity A,1970-01-01 09:00:00,Resource_Group2_res_1,normal
1,case_0,Activity B,1970-01-01 10:00:00,Resource_Group0_res_0,normal
2,case_0,Activity C,1970-01-01 11:00:00,Resource_Group1_res_0,normal
3,case_0,Activity D,1970-01-01 12:00:00,Resource_Group2_res_2,normal
4,case_0,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,normal
...,...,...,...,...,...
45842,case_999,Activity E,1970-01-01 13:00:00,Resource_Group0_res_0,normal
45843,case_999,Activity K,1970-01-01 14:00:00,Resource_Group1_res_1,normal
45844,case_999,Activity Q,1970-01-01 15:00:00,Resource_Group0_res_0,normal
45845,case_999,Activity L,1970-01-01 16:00:00,Resource_Group0_res_0,normal


 Create a function to extract the prefixes of length L from the event log.

In [4]:
def create_prefix(df, L):
    res0 = df.sort_values(by=['caseid','ts']).groupby('caseid', as_index=False)['activity'].apply(list)    
    res1 = df.sort_values(by=['caseid','ts']).groupby('caseid', as_index=False)['ts'].apply(list)
    res2 = df.sort_values(by=['caseid','ts']).groupby('caseid', as_index=False)['resource'].apply(list)
    
    res = pd.merge(res0, pd.merge(res1, res2, on="caseid") , on="caseid")
    res["outcome"] = df["outcome"]

    
    event_num = df.groupby('caseid')['activity'].count().to_frame('event_number')
    res = pd.merge(res, event_num, on="caseid")
    
    res[res["event_number"] < L] = np.nan
    res = res.dropna()
    
    def function(row, label):
        activity_array = row[label][0:L]
        return activity_array
    
    for c in ["activity", "ts", "resource"]:
        res[c] = res.apply(lambda row: function(row, c), axis=1)
    
    res = res.drop(columns=["event_number"])
    return res
    

In [72]:
x = create_prefix(df, 11)
x

,caseid,activity,ts,resource,outcome
1,case_1,"[Activity A, Activity B, Activity C, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,...",normal
7,case_1003,"[Activity A, Activity B, Activity C, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,...",insert
10,case_1006,"[Activity A, Activity B, Activity M, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_2, Resource_Group0_res_0,...",insert
29,case_1023,"[Activity A, Activity S, Activity R, Activity ...","[1970-01-01 09:00:00, 1970-01-01 09:28:09, 197...","[Resource_Group2_res_2, Resource_Group1_res_1,...",normal
45,case_1038,"[Activity A, Activity B, Activity C, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,...",normal
...,...,...,...,...,...
4941,case_946,"[Activity A, Activity B, Activity C, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,...",normal
4944,case_949,"[Activity A, Activity B, Activity C, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_1, Resource_Group0_res_0,...",normal
4956,case_96,"[Activity A, Activity B, Activity C, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_0, Resource_Group0_res_0,...",normal
4965,case_968,"[Activity A, Activity B, Activity C, Activity ...","[1970-01-01 09:00:00, 1970-01-01 10:00:00, 197...","[Resource_Group2_res_2, Resource_Group0_res_0,...",normal


Create a function that does aggregation encoding of a prefix extracted from the event log

In [114]:
def one_hot(x, label, fun):
    unique_val = []
    #Extract unique values of label
    for element in x[label]:
        for e in element:
            if e not in unique_val:
                unique_val.append(e)
    
    result = x[["caseid"]].copy()
    
    if fun == "index":
        #Create columns
        L = len(x[label][1]) # Length of the prefix
        for e in unique_val:
            for i in range(1, L+1): 
                result[label + "_" + str(i)+ "_" + e] = 0
        
        for index, row in x.iterrows():
            k = 1
            for e in row[label]:
                result.at[index, label + "_" + str(k) + "_" + e] += 1
                k +=1
    
    else:
        #Create columns
        for t in unique_val:
            result[t] = 0
        
        #Add one to the right column if value is in row
        for index, row in x.iterrows():
            for element in row[label]:
                result.at[index, element] += 1

    return result

def average_timestamp(df, label):

    result = df[["caseid"]].copy()
    result["avg_"+label] = 0

    L = len(df[label][1])
    
    for index, row in df.iterrows():
        temp = 0
        for element in row[label]:
            temp += calendar.timegm(datetime.strptime(element, '%Y-%m-%d %H:%M:%S').timetuple())
        result.at[index, "avg_" + label] = time.strftime('%Y-%m-%d %H:%M:%S', time.gmtime(temp/L))
    
    return result


In [115]:
average_timestamp(x, "ts")

,caseid,avg_ts
0,case_0,1970-01-01 11:00:00
1,case_1,1970-01-01 10:35:44
2,case_10,1970-01-01 11:00:00
3,case_100,1970-01-01 11:00:00
4,case_1000,1970-01-01 11:00:00
...,...,...
4995,case_995,1970-01-01 11:00:00
4996,case_996,1970-01-01 11:00:00
4997,case_997,1970-01-01 11:00:00
4998,case_998,1970-01-01 11:00:00


In [111]:
one_hot(x, "activity", "aggr")

,caseid,Activity A,Activity B,Activity C,Activity D,Activity E,Activity G,Activity R,Activity O,Activity P,Activity M,Activity S,Activity K,Activity Q,Activity I,Activity N,Activity L,Activity J,Activity H,Activity T
0,case_0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,case_1,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
2,case_10,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,case_100,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,case_1000,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,case_995,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4996,case_996,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4997,case_997,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4998,case_998,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [84]:
one_hot(x, "resource", "aggr")

,caseid,Resource_Group2_res_0,Resource_Group0_res_0,Resource_Group1_res_1,Resource_Group2_res_1,Resource_Group1_res_0,Resource_Group2_res_2
1,case_1,1,5,3,2,0,0
7,case_1003,1,3,2,2,3,0
10,case_1006,1,6,1,0,1,2
29,case_1023,1,5,2,0,2,1
45,case_1038,2,6,1,1,1,0
...,...,...,...,...,...,...,...
4941,case_946,1,4,2,2,2,0
4944,case_949,2,5,2,1,1,0
4956,case_96,1,3,1,0,4,2
4965,case_968,0,3,3,0,2,3


In [87]:
def aggregation_enc(df):
    res0 = one_hot(df, "activity", "aggr")
    res1 = one_hot(df, "resource", "aggr")
    res2 = average_timestamp(df, "ts")
    
    res = pd.merge(res0, pd.merge(res1, res2), on="caseid")
    
    temp = df[["caseid", "outcome"]].copy()
    res = pd.merge(res, temp, on="caseid")

    return res

In [88]:
aggregation_enc(x)

,caseid,Activity A,Activity B,Activity C,Activity G,Activity R,Activity D,Activity E,Activity K,Activity M,...,Activity J,Activity T,Resource_Group2_res_0,Resource_Group0_res_0,Resource_Group1_res_1,Resource_Group2_res_1,Resource_Group1_res_0,Resource_Group2_res_2,avg_ts,outcome
0,case_1,1,1,1,1,1,1,1,1,1,...,0,0,1,5,3,2,0,0,1970-01-01 12:43:31,normal
1,case_1003,1,1,1,1,0,1,1,0,0,...,0,0,1,3,2,2,3,0,1970-01-01 12:11:50,insert
2,case_1006,1,1,1,1,0,1,1,1,1,...,0,0,1,6,1,0,1,2,1970-01-01 12:30:13,insert
3,case_1023,1,1,1,0,1,1,1,1,0,...,0,0,1,5,2,0,2,1,1970-01-01 12:22:11,normal
4,case_1038,1,1,1,1,0,1,1,1,1,...,0,0,2,6,1,1,1,0,1970-01-01 13:43:38,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,case_946,1,1,1,1,0,1,1,0,0,...,0,0,1,4,2,2,2,0,1970-01-01 12:31:20,normal
441,case_949,1,1,1,1,0,1,1,0,1,...,0,1,2,5,2,1,1,0,1970-01-01 12:44:33,normal
442,case_96,1,1,1,1,0,1,1,1,0,...,0,0,1,3,1,0,4,2,1970-01-01 12:31:22,normal
443,case_968,1,1,1,1,1,1,1,1,0,...,0,0,0,3,3,0,2,3,1970-01-01 13:45:51,normal


Create a function that does index-based encoding of a prefix extracted from the event log.

In [100]:
def index_label(df, label):

    L = len(df[label][1])
    result = x[["caseid"]].copy()
    
    # Create the column and put them to 0
    for i in range(1, L+1):
        result[label + "_" + str(i)] = 0
        
    for index, row in x.iterrows():
        k = 1
        for e in row[label]:
            result.at[index, label + "_" + str(k)] = e
            k +=1
    
    return result

In [90]:
one_hot(x, "activity", "index")

,caseid,activity_1_Activity A,activity_2_Activity A,activity_3_Activity A,activity_4_Activity A,activity_5_Activity A,activity_6_Activity A,activity_7_Activity A,activity_8_Activity A,activity_9_Activity A,...,activity_2_Activity T,activity_3_Activity T,activity_4_Activity T,activity_5_Activity T,activity_6_Activity T,activity_7_Activity T,activity_8_Activity T,activity_9_Activity T,activity_10_Activity T,activity_11_Activity T
1,case_1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,case_1003,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,case_1006,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,case_1023,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,case_1038,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941,case_946,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4944,case_949,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4956,case_96,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4965,case_968,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [91]:
one_hot(x, "resource", "index")

,caseid,resource_1_Resource_Group2_res_0,resource_2_Resource_Group2_res_0,resource_3_Resource_Group2_res_0,resource_4_Resource_Group2_res_0,resource_5_Resource_Group2_res_0,resource_6_Resource_Group2_res_0,resource_7_Resource_Group2_res_0,resource_8_Resource_Group2_res_0,resource_9_Resource_Group2_res_0,...,resource_2_Resource_Group2_res_2,resource_3_Resource_Group2_res_2,resource_4_Resource_Group2_res_2,resource_5_Resource_Group2_res_2,resource_6_Resource_Group2_res_2,resource_7_Resource_Group2_res_2,resource_8_Resource_Group2_res_2,resource_9_Resource_Group2_res_2,resource_10_Resource_Group2_res_2,resource_11_Resource_Group2_res_2
1,case_1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,case_1003,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,case_1006,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
29,case_1023,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45,case_1038,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941,case_946,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4944,case_949,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4956,case_96,1,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
4965,case_968,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [101]:
index_label(x, "ts")

,caseid,ts_1,ts_2,ts_3,ts_4,ts_5,ts_6,ts_7,ts_8,ts_9,ts_10,ts_11
1,case_1,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 11:07:35,1970-01-01 11:51:06,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00
7,case_1003,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 11:07:02,1970-01-01 11:08:41,1970-01-01 11:54:37,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00
10,case_1006,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 10:15:08,1970-01-01 10:17:19,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00
29,case_1023,1970-01-01 09:00:00,1970-01-01 09:28:09,1970-01-01 09:35:59,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00
45,case_1038,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00,1970-01-01 17:00:00,1970-01-01 17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
4941,case_946,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 12:06:28,1970-01-01 12:48:04,1970-01-01 12:50:18,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00
4944,case_949,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 13:02:28,1970-01-01 13:15:42,1970-01-01 13:51:57,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00
4956,case_96,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 12:04:55,1970-01-01 12:46:46,1970-01-01 12:53:21,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00
4965,case_968,1970-01-01 09:00:00,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00,1970-01-01 17:11:20,1970-01-01 17:13:07


In [102]:
def index_enc(df):
    res0 = one_hot(df, "activity", "index")
    res1 = one_hot(df, "resource", "index")
    res2 = index_label(df, "ts")
    
    res = pd.merge(res0, pd.merge(res1, res2), on="caseid")
    
    temp = df[["caseid", "outcome"]].copy()
    res = pd.merge(res, temp, on="caseid")

    return res

In [103]:
index_enc(x)

,caseid,activity_1_Activity A,activity_2_Activity A,activity_3_Activity A,activity_4_Activity A,activity_5_Activity A,activity_6_Activity A,activity_7_Activity A,activity_8_Activity A,activity_9_Activity A,...,ts_3,ts_4,ts_5,ts_6,ts_7,ts_8,ts_9,ts_10,ts_11,outcome
0,case_1,1,0,0,0,0,0,0,0,0,...,1970-01-01 11:00:00,1970-01-01 11:07:35,1970-01-01 11:51:06,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00,normal
1,case_1003,1,0,0,0,0,0,0,0,0,...,1970-01-01 11:00:00,1970-01-01 11:07:02,1970-01-01 11:08:41,1970-01-01 11:54:37,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,insert
2,case_1006,1,0,0,0,0,0,0,0,0,...,1970-01-01 10:15:08,1970-01-01 10:17:19,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00,insert
3,case_1023,1,0,0,0,0,0,0,0,0,...,1970-01-01 09:35:59,1970-01-01 10:00:00,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00,normal
4,case_1038,1,0,0,0,0,0,0,0,0,...,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00,1970-01-01 17:00:00,1970-01-01 17:00:00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,case_946,1,0,0,0,0,0,0,0,0,...,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 12:06:28,1970-01-01 12:48:04,1970-01-01 12:50:18,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,normal
441,case_949,1,0,0,0,0,0,0,0,0,...,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 13:02:28,1970-01-01 13:15:42,1970-01-01 13:51:57,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,normal
442,case_96,1,0,0,0,0,0,0,0,0,...,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 12:04:55,1970-01-01 12:46:46,1970-01-01 12:53:21,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,normal
443,case_968,1,0,0,0,0,0,0,0,0,...,1970-01-01 11:00:00,1970-01-01 12:00:00,1970-01-01 13:00:00,1970-01-01 14:00:00,1970-01-01 15:00:00,1970-01-01 16:00:00,1970-01-01 17:00:00,1970-01-01 17:11:20,1970-01-01 17:13:07,normal


Train a decision tree (DT) and a randomforest (RF) from the scikit-learn package for outcome label prediction for the event log for prefix lengths L=2 and L=5.  Split 70-30 for training-testing (no temporal splitting or cross-validation needed!).

In [104]:
def classification(X, y, classifier, seed):
    X.astype(int)

    X = np.nan_to_num(X)
    clf = RandomForestClassifier()

    if classifier == "RF":
        clf = RandomForestClassifier(n_estimators=100, oob_score=True)
    elif classifier == "DT":
        clf = DecisionTreeClassifier(max_depth=5)

    print("Training for "+classifier+"...")

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print("Precision: "+str(precision))
    print("Recall: "+str(recall))
    print("F1: "+str(f1))

    return {"precision": precision,
            "recall": recall,
            "f1": f1}

In [105]:
x = create_prefix(df, 2)
x_encoded = one_hot(x, "activity", "aggr")
y = x["outcome"]
columns = x_encoded.columns
columns = x_encoded.columns[1:len(columns)]
classification(x_encoded[columns], y, "RF", 1)

Training for RF...
Precision: 0.7592217777777778
Recall: 0.8713333333333333
F1: 0.8114233463959151


{'precision': 0.7592217777777778,
 'recall': 0.8713333333333333,
 'f1': 0.8114233463959151}

In [106]:
x = create_prefix(df, 5)
x_encoded = one_hot(x, "activity", "aggr")
y = x["outcome"]
columns = x_encoded.columns
columns = x_encoded.columns[1:len(columns)]
classification(x_encoded[columns], y, "RF", 1)

Training for RF...
Precision: 0.7966252671195739
Recall: 0.8693333333333333
F1: 0.8129133333333333


{'precision': 0.7966252671195739,
 'recall': 0.8693333333333333,
 'f1': 0.8129133333333333}

In [107]:
x = create_prefix(df, 2)
x_encoded = one_hot(x, "activity", "aggr")
y = x["outcome"]
columns = x_encoded.columns
columns = x_encoded.columns[1:len(columns)]
classification(x_encoded[columns], y, "DT", 1)

Training for DT...
Precision: 0.8024680472055221
Recall: 0.8706666666666667
F1: 0.8123614273098683


{'precision': 0.8024680472055221,
 'recall': 0.8706666666666667,
 'f1': 0.8123614273098683}

In [108]:
x = create_prefix(df, 5)
x_encoded = one_hot(x, "activity", "aggr")
y = x["outcome"]
columns = x_encoded.columns
columns = x_encoded.columns[1:len(columns)]
classification(x_encoded[columns], y, "DT", 1)


Training for DT...
Precision: 0.7966252671195739
Recall: 0.8693333333333333
F1: 0.8129133333333333


{'precision': 0.7966252671195739,
 'recall': 0.8693333333333333,
 'f1': 0.8129133333333333}